In [ ]:
import numpy as np
import modin.pandas as pd

In [ ]:
load_columns = [
    "Index",
    "ph",
    "iron",
    "nitrate",
    "chloride",
    "lead",
    "zinc",
    "color",
    "turbidity",
    "fluoride",
    "copper",
    "odor",
    "sulfate",
    "conductivity",
    "chlorine",
    "manganese",
    "total_dissolved_solids",
    "source",
    "water_temperature",
    "air_temperature",
    "month",
    "day",
    "time_of_day",
    "target",
]

df = pd.read_csv("../../dataset/freshwater/dataset.csv", index_col="Index", names=load_columns, skiprows=1)
df.info()

In [ ]:
df.iloc[0, :]

In [ ]:
df["date"] = df.apply(lambda x: f"2023-{x.month}-{x.day:02,.0f} {x.time_of_day:02,.0f}:00:00" if not pd.isna(x.month) and not pd.isna(x.day) and not pd.isna(x.time_of_day) else np.nan, axis=1)
df["date"] = pd.to_datetime(df["date"])

df.to_parquet("../../dataset/freshwater/dataset-p2.parquet")

In [ ]:
df_missing_values = df.isna().sum() / df.shape[0] * 100
df_missing_values.plot(kind="bar")

In [ ]:
df_ts = df.dropna(subset=["date"])
df_ts = df_ts.set_index("date")
df_ts = df_ts.fillna(0)
df_ts = df_ts.resample("W").mean()
df_ts.to_json("../app_web/assets/data/water-chemicals.json", orient="index")

df_ts.plot(figsize=(20, 10))

In [ ]:
df_count_color = df["color"].value_counts().sort_values()
df_count_color.to_json("../app_web/assets/data/water-color.json", orient="split")
df_count_color

In [ ]:
df_count_source = df["source"].value_counts().sort_values()
df_count_source.to_json("../app_web/assets/data/water-source.json", orient="split")
df_count_source

In [ ]:
df_count_target = df["target"].value_counts().sort_index()
df_count_target = df_count_target.rename(index={0:"Good", 1:"Bad"})
df_count_target.to_json("../app_web/assets/data/water-quality.json", orient="split")
df_count_target